In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


In [2]:
import folium
import requests
from datetime import datetime
import ee
from folium import plugins
from folium.plugins import minimap

In [3]:
# Criar o mapa Folium
my_map = folium.Map(location=[-13.531950, -54.791014], zoom_start=6)

# Requisição de Dados Metereologicos

API_KEY = "d2f43db503fef82f2052e803c234e2f2"

# Coordenadas das cidades
cidades = {
    'Água Boa': (-14.0504, -52.1606),
    'Arenápolis': (-14.4473, -56.8531),
    'Barão de Melgaço': (-16.2060, -55.9671),
    'Barra do Garças': (-15.8833, -52.2569),
    'Cáceres': (-16.0735, -57.6819),
    'Campo Novo do Parecis': (-13.6681, -57.8903),
    'Chapada dos Guimarães': (-15.4649, -55.7497),
    'Comodoro': (-13.6618, -59.7847),
    'Confresa': (-10.6346, -51.5696),
    'Cuiabá': (-15.5989, -56.0949),
    'Feliz Natal': (-12.3840, -54.9223),
    'Nova Canaã do Norte': (-10.5580, -55.7276),
    'Nova Olímpia': (-14.7958, -57.2880),
    'Paranatinga': (-14.4265, -54.0510),
    'Peixoto de Azevedo': (-10.2269, -54.9794),
    'Pontes de Lacerda': (-15.2290, -59.3381),
    'Poxoréu': (-15.8295, -54.4011),
    'Rondonópolis': (-16.4676, -54.6372),
    'Santa Terezinha': (-10.4704, -50.5141),
    'Santo Antônio do Leverger': (-15.8653, -56.0789),
    'São José do Rio Claro': (-13.4606, -56.7216),
    'Vila Rica': (-10.0192, -51.1189),
    'Tangará da Serra': (-14.6229, -57.5000),
    'Sorriso': (-12.5422, -55.7219),
    'Guarantã do Norte': (-9.97063, -54.9049),
    'Alto Araguaia': (-17.3164, -53.2199),
    'Apiacás': (-9.54007, -57.4596),
    'Sapezal': (-13.5403, -58.2441),
    'Campo Verde': (-15.5408, -55.1606),
    'Juara': (-11.2634, -57.5242),
    'Querência': (-12.6102, -52.1826),
    'Sinop': (-11.8668, -55.5081),
    'Cotriguaçu': (-9.85753, -58.4198),
    'Juína': (-11.3728, -58.7489),
    'São Félix do Araguaia': (-11.6144, -50.6704),
    'Vila Bela da Santíssima Trindade': (-15.0067, -59.9500),
    'Primavera do Leste': (-15.5950, -54.2990),
    'Alta Floresta': (-9.8697, -56.0862),
    'Carlinda': (-9.94915, -55.8415),
    'Brasnorte': (-12.1472, -57.9833),
    'Nova Maringá': (-13.0136, -57.0907),
    'Nova Ubiratã': (-12.9830, -55.2567),
    'Gaúcha do Norte': (-13.2427, -53.0809),
    'Santo Antônio do Leste': (-14.8058, -53.6155),
    'Guiratinga': (-16.3466, -53.7576),
    'Itiquira': (-17.2147, -54.1425),
    'Alto Taquari': (-17.8350, -53.2792),
    'Porto Estrela': (-15.3230, -57.2204),
    'Salto do Céu': (-15.1307, -58.1323),
    'São José do Xingu': (-10.7983, -52.7486),
    'Serra Nova Dourada': (-12.0899, -51.7659),
    'Rosário Oeste': (-14.8406, -56.4307),
    'Canarana': (-13.5512, -52.2705),
    'Nova Xavantina': (-14.6774, -52.3506),
    'Serra Nova Dourada': (-12.0899, -51.7659),
    'Nova Ubiratã': (-12.9830, -55.2567),
    'Diamantino': (-14.4032, -56.4379)
}

# Adicionar grupo de marcadores para as cidades
city_markers = folium.FeatureGroup(name='Cidades')
my_map.add_child(city_markers)

# Adicionar marcadores para as cidades e dados do clima e umidade
for cidade, coordenadas in cidades.items():
    lat, lon = coordenadas
    link = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_KEY}&lang=pt"
    requisicao = requests.get(link)
    requisicao_dic = requisicao.json()

    if 'weather' in requisicao_dic:
        descricao = requisicao_dic['weather'][0].get('description', "Descrição do tempo indisponível")
    else:
        descricao = "Descrição do tempo indisponível"

    if 'main' in requisicao_dic:
        temperatura = requisicao_dic['main'].get('temp', 0) - 273.15
        umidade = requisicao_dic['main'].get('humidity', 0)
    else:
        temperatura = 0
        umidade = 0

    if temperatura > 38:
        icone = folium.Icon(color='red')
    elif 'chuva' in descricao.lower():
        icone = folium.Icon(color='orange')  # Alterado de 'yellow' para 'orange'        
    else:
        icone = folium.Icon(color='blue')

    pop_up = f"Cidade: {cidade}<br>Descrição: {descricao}<br>Temperatura: {temperatura:.2f}°C<br>Umidade: {umidade}%"
    folium.Marker(coordenadas, icon=icone, popup=folium.Popup(pop_up, parse_html=True)).add_to(city_markers)

# Adicionar controle de legenda ao mapa
legend_html = """
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; height: 80px; 
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color: white; opacity: 0.85;">
    &nbsp; Legenda <br>
    &nbsp; <i class="fa fa-circle fa-1x" style="color:red"></i> Temperatura > 38°C <br>
    &nbsp; <i class="fa fa-circle fa-1x" style="color:orange"></i> Chuva <br>
    &nbsp; <i class="fa fa-circle fa-1x" style="color:blue"></i> Outras condições <br>
    </div>
"""

my_map.get_root().html.add_child(folium.Element(legend_html));

# Importar a coleção de dados FIRMS.

dataset = ee.ImageCollection('FIRMS').filter(
    ee.Filter.date(datetime(2023, 11, 22), datetime(2023, 11, 23)))

# Selecionar a banda que contém os focos de incêndio.
fires = dataset.select('T21')

# Clip (recortar) os focos de incêndio para o estado de Mato Grosso.
matoGrossoBorder = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(ee.Filter.eq('ADM1_NAME', 'Mato Grosso'))
firesInMatoGrosso = fires.map(lambda image: image.clip(matoGrossoBorder))

# Adicionar grupo de marcadores para os focos de incêndio
fire_markers = folium.FeatureGroup(name='Focos de Incêndio em Mato Grosso')
my_map.add_child(fire_markers)

# Definir a visualização dos focos de incêndio.
firesVis = {
    'min': 325.0,
    'max': 400.0,
    'palette': ['red', 'orange', 'yellow']
}

# Adicionar a camada de focos de incêndio ao mapa.
fires_layer = folium.TileLayer(
    tiles=firesInMatoGrosso.getMapId(firesVis)['tile_fetcher'].url_format,
    attr='Focos de Incêndio em Mato Grosso',
    name='Focos de Incêndio em Mato Grosso'
).add_to(fire_markers)

#DADOS DE CHUVAS

# GPM V6 30 minute data around hurricane Dorian for a single day.
date_range = ee.Date(datetime(2023, 11, 23)).getRange('day')
dataset = ee.ImageCollection('NASA/GPM_L3/IMERG_V06').filter(ee.Filter.date(date_range))

# Selecionar a precipitação máxima e mascarar valores de baixa precipitação.
precipitation = dataset.select('precipitationCal').max()
mask = precipitation.gt(0.5)
precipitation = precipitation.updateMask(mask)

# Definir a paleta de cores
palette = ['000096','0064ff', '00b4ff', '33db80', '9beb4a', 'ffeb00', 'ffb300', 'ff6400', 'eb1e00', 'af0000']
precipitation_vis = {'min': 0, 'max': 15, 'palette': palette}

# Adicionar a camada de precipitação ao mapa usando a biblioteca folium
folium.TileLayer(
    tiles=precipitation.getMapId(precipitation_vis)['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='Precipitation (mm/hr)'
).add_to(my_map)

# Adicionar controle de camadas ao mapa

folium.LayerControl().add_to(my_map);



In [4]:
# Adicionar minimapa ao mapa (corrigir o nome da variável)
minimapa = folium.plugins.MiniMap(tile_layer="cartodbpositron", position="bottomright")
my_map.add_child(minimapa)

# Exibir o mapa
display(my_map)

# Salvar o mapa como um arquivo HTML na pasta "templates"
html_file_path = "templates/mapa.html"
my_map.save(html_file_path)
print(f"Mapa salvo em: {html_file_path}")


Mapa salvo em: templates/mapa.html
